In [8]:
import os
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

response = client.files.create(
  file=open("articles_trimmed_new_1.jsonl", "rb"),
  purpose="fine-tune"
)

In [6]:
'''FileObject(id='file-sUjBkhraDYSKK4VzIdGMevTo', bytes=3302, created_at=1729191039, filename='toy_chat_fine_tuning.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)'''

"FileObject(id='file-sUjBkhraDYSKK4VzIdGMevTo', bytes=3302, created_at=1729191039, filename='toy_chat_fine_tuning.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)"

In [16]:
print(response)

FileObject(id='file-5Idid2RIzncv3kjEBOljfhVa', bytes=275992760, created_at=1729217629, filename='articles_trimmed_new_1.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)


In [10]:
import json
import tiktoken # for token counting
import numpy as np
from collections import defaultdict

data_path = "articles_trimmed_new_1.jsonl"

# Load the dataset
with open(data_path, 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)

Num examples: 50000
First example:
{'role': 'system', 'content': 'Provide 1-2 line titles for articles'}
{'role': 'user', 'content': 'AGILE DEVELOPMENT\n\nTeleconferencing Is the Pair Programming of the Future\n\nCollaboration for developers has never been easier.\n\nCoding via teleconference is similar to the situation above, but nobody has to shower!\n\nMy challenge to you…\n\nIf you’re a leading member of a development team that typically works separately, I triple-dog-dare you to:\n\nPick a task that you think will take two weeks for your development team to complete (working separately). Perhaps an upcoming backlog item. Invite your entire team to a call on MS Teams, Zoom, or whichever teleconferencing tool you use. Have them drop as many meetings as possible so that they’re free for all or most of the day. Start sharing your screen, talk about what you’re trying to do, and start coding while the rest of the team is watching. Start asking a few questions. Don’t force any structure

In [11]:
# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue
        
    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue
        
    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1
        
        if any(k not in ("role", "content", "name", "function_call", "weight") for k in message):
            format_errors["message_unrecognized_key"] += 1
        
        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            format_errors["unrecognized_role"] += 1
            
        content = message.get("content", None)
        function_call = message.get("function_call", None)
        
        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1
    
    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

No errors found


In [18]:
import os
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

job = client.fine_tuning.jobs.create(
  training_file="file-5Idid2RIzncv3kjEBOljfhVa",
  model="gpt-4o-mini-2024-07-18"
)

In [20]:
job

FineTuningJob(id='ftjob-5a1NUef4yMKKN3z1xAg7ooYj', created_at=1729218486, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-k2PeUErkTQUe1i20vWc6OlmA', result_files=[], seed=992869779, status='validating_files', trained_tokens=None, training_file='file-5Idid2RIzncv3kjEBOljfhVa', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)

In [26]:
client.fine_tuning.jobs.retrieve("ftjob-5a1NUef4yMKKN3z1xAg7ooYj")

FineTuningJob(id='ftjob-5a1NUef4yMKKN3z1xAg7ooYj', created_at=1729218486, error=Error(code='exceeded_quota', message='Creating this fine-tuning job would exceed your hard limit, please check your plan and billing details.                     Cost of job ftjob-5a1NUef4yMKKN3z1xAg7ooYj: USD 167.43. Quota remaining for org-k2PeUErkTQUe1i20vWc6OlmA: USD 116.38.', param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=1, batch_size=33, learning_rate_multiplier=1.8), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-k2PeUErkTQUe1i20vWc6OlmA', result_files=[], seed=992869779, status='failed', trained_tokens=None, training_file='file-5Idid2RIzncv3kjEBOljfhVa', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)

In [20]:
'''FineTuningJob(id='ftjob-74ft7Aat5zCLN2A6oEKAnA38', created_at=1729195628, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=2, learning_rate_multiplier=1.8), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-k2PeUErkTQUe1i20vWc6OlmA', result_files=[], seed=1183855914, status='running', trained_tokens=None, training_file='file-lqAKQ1g061cWZKrUoE7FArmW', validation_file=None, estimated_finish=1729197559, integrations=[], user_provided_suffix=None)'''

"FineTuningJob(id='ftjob-74ft7Aat5zCLN2A6oEKAnA38', created_at=1729195628, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=2, learning_rate_multiplier=1.8), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-k2PeUErkTQUe1i20vWc6OlmA', result_files=[], seed=1183855914, status='running', trained_tokens=None, training_file='file-lqAKQ1g061cWZKrUoE7FArmW', validation_file=None, estimated_finish=1729197559, integrations=[], user_provided_suffix=None)"

In [24]:
'''FineTuningJob(id='ftjob-74ft7Aat5zCLN2A6oEKAnA38', created_at=1729195628, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=2, learning_rate_multiplier=1.8), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-k2PeUErkTQUe1i20vWc6OlmA', result_files=[], seed=1183855914, status='running', trained_tokens=None, training_file='file-lqAKQ1g061cWZKrUoE7FArmW', validation_file=None, estimated_finish=1729197661, integrations=[], user_provided_suffix=None)'''

"FineTuningJob(id='ftjob-74ft7Aat5zCLN2A6oEKAnA38', created_at=1729195628, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=2, learning_rate_multiplier=1.8), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-k2PeUErkTQUe1i20vWc6OlmA', result_files=[], seed=1183855914, status='running', trained_tokens=None, training_file='file-lqAKQ1g061cWZKrUoE7FArmW', validation_file=None, estimated_finish=1729197661, integrations=[], user_provided_suffix=None)"